# back test 回测程序

## bt_list : 回测清单



In [115]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams["font.family"] = 'Arial Unicode MS'
# mpl.rcParams["font.sans-serif"] = ["Arial Unicode MS"]
mpl.rcParams["axes.unicode_minus"] = False
# import mplfinance as mpf

from importlib import reload 
from qflib import basic,backtest
reload(basic)
engine = basic.engine()

data_dir='../'
# data_dir='~/qfdata/'


In [116]:
# 初始化 bt_list
''' 测试清单 '''
# bt_file = '../data/sl_shuangtoupao.csv'   # 多头炮清单
# bt_file = '../data/sl_company.csv'
bt_file = '../data/stocklist.csv'
bt_list = pd.read_csv(bt_file)

# bt_list.to_csv(bt_file, index=False)

bt_list['st_s1'] = 0
start_date = '2020-01-01'
end_date= '2022-12-31'
skip_days = 10

# print( bt_list )

# 初始化bt_result, 无记录
bt_result = pd.DataFrame(columns=['ts_code', 'start_date', 'end_date','pct_profit','pct_noaction','pct_diff'])
# print(bt_result)


In [117]:
%%time
reload(backtest)
for i in range(0, len(bt_list) ):
    
    ts_code = bt_list.loc[i,'ts_code']
    if ts_code.startswith('8') or ts_code.startswith('688') :
        continue
    
    # 1、csv 读取
    ts_cal_file = data_dir+'cal/' + ts_code+'.cal.csv'     
    df = pd.read_csv(ts_cal_file, index_col='trade_date', parse_dates=True)
    # 2、mysql 读取
    # sql = "select * from stock_cal where ts_code='" + ts_code + "'"
    # df = pd.read_sql_query(sql, engine.connect(), index_col='trade_date')
    # 3、Pandas 内存读取
    # df = dfcal[ dfcal.index=='601318.SH' ].copy()  
    # df.sort_values(by='trade_date', ascending=True)

    # print( df )
    # break

    # 计算买卖点
    df['bt_s1'] = 0  # View显示字段, 1类买卖点  
    df['bt_dtts'], df['bt_xrds'], df['bt_qmzx'], df['bt_chuizi'] = 0, 0, 0, 0  # 多头吞噬, 旭日东升, 启明之星，锤子Plus 
    df['bt_zonghe']  = 0  # charlie: 综合 
    df['bt_stp'] = 0  # 双头炮1 and 20

    # 涨停后倍量长阴法: run_blcy:
    # run_cydjx : 穿越多均线气动法

    # df['pct_st_15d_max']=df['pct_st_range'].rolling(15).max()
    buy_day, sale_day = [], []
    run_xrds, run_dtts, run_qmzx, run_chuizi = 0,0,0,0
    # run_xrds, run_dtts, run_qmzx, run_chuizi = 0,0,1,1
    # run_xrds, run_dtts, run_qmzx, run_chuizi = 1,1,1,1
    run_stp, run_stp20, run_stp30 = 0,1,0
    run_blcy, run_cydjx = 0,0

    run_zonghe = 0 

    # 旭日东升模式 xrdf 
    # 連續3天下跌、今天高开、高收、中阳以上
    if run_xrds == True:
        res = backtest.xrds(df, skip_days, 5)  
        # def : 今日涨幅 
        buy_day += res[0]

    # 多头吞噬法买， 空头吞噬法卖出
    # 低开高走：今天实体低点低于昨天低点， 高点高于昨天高点； 中阳以上
    if run_dtts == True:
        res = backtest.dtts(df, skip_days, 5)  
        # dtts(df, pct_change_td-今日涨幅)
        buy_day += res[0]
        
    # 启明之星模式 qmzx 
    # 启明之星 = 前日阴线 + 实体缺口 + 星线 + 今日的多头吞噬(低开高走) 或 旭日东升(高开高走)
    if run_qmzx == True:
        res = backtest.qmzx(df, skip_days, 4)
        buy_day += res[0]

    # 锤子 + 倒锤子
    if run_chuizi == 1: 
        res = backtest.chuizi(df, skip_days, -1, 4, 4, 2)   # 昨日长下引线+今日中阳+短上引线
        # df, 前3/5/10日平均跌幅>，昨日下引线>，今日涨幅>, 今日上引线<
        # ys: yesterday; td: today
        buy_day += res[0]

    # 综合 - Charlie
    if run_zonghe == 1:
        # res = backtest.czkz(df, -1.5, 2, 4, 8)    # 连续下跌后反弹明显
        res = backtest.zonghe(df, skip_days, 0, 4, 0, 0)     # 昨日长下引线 + 前10天大阳线
        # df, 前3日平均跌幅，昨日下引线，今日涨幅, 昨日今日2日振幅
        buy_day += res[0]

    # 双头跑
    if run_stp == 1:
        res = backtest.stp(df, skip_days,    9,           5,         15)   
        #                  df, 往前运算天数， 板涨幅，中间小阳小阴幅度，双涨之间最大天数
        buy_day += res[0]
        # df = res[0]

    # 双头跑20
    if run_stp20 == 1:
        res = backtest.stp20(df, )   
        #                  df, 往前运算天数， 板涨幅，中间小阳小阴幅度，双涨之间最大天数
        buy_day += res[0]
        # df = res[0]

    # 涨停倍量长阴洗盘结束法
    if run_blcy == 1:
        res = backtest.blcy(df)   
        #                  df, 往前运算天数， 板涨幅，中间小阳小阴幅度，双涨之间最大天数
        buy_day += res[0]
        # df = res[0]

    # 穿越多均线启动法
    if run_cydjx == 1:
        res = backtest.cydjx(df)   
        #                  df, 往前运算天数， 板涨幅，中间小阳小阴幅度，双涨之间最大天数
        buy_day += res[0]
        # df = res[0]
    
    df['bt_s1']  = df.apply(backtest.merge_actions, axis=1)
    # 存在买点或卖点
    if buy_day or sale_day:
        print( '\n', i, ts_code, buy_day)
        # print( df[['pct_change','xing','bt_s1','bt_stp']][ df['bt_s1']==1 ] )

        df.to_csv( data_dir+'/bt/' + ts_code + '.bt.csv' )
        bt_list.loc[i,'bt_s1'] = 1
        # bt_list.loc[i,'st_s1'] = 1
    else:
        print( ' . ', end='   ' )

df_viewlist = bt_list[ bt_list['bt_s1']==1 ]
df_viewlist.to_csv('../data/viewlist.csv', index=False)
df_viewlist.to_csv('../data/viewlist.txt', index=False)
print('\n=============================')
print('操作数量：', df_viewlist.shape[0])

    # df.loc[df.index[i], 'bt_stp'] = 1
    
    # print( '' )
    # ''' end 0.5 '''
       
    # 计算赢率
    # ts_log_file = '../bt/' + ts_code+'.log.csv'  # print(df_log)
    # df_log = bt_profit(df, df_bt)
    # df_log.to_csv(ts_log_file)
    # # print(df_log)
    # # 回测结果总表
    # # bt_result = pd.DataFrame(columns=['ts_code', 'start_date', 'end_date','pct_profit','pct_noaction','pct_diff'])
    # # 不操作利润
    # pct_profit = df_log.loc[ df_log.index[-1], 'pct_profit' ]
    # pct_noaction = (df.loc[ df.index[-1], 'open' ] - df.loc[ df.index[0], 'open'])/df.loc[ df.index[0], 'open'] * 100
    # # 输出
    # line0 = pd.Series({'ts_code':ts_code, 'start_date':start_date, 'end_date':end_date, 
    #     'pct_profit':round(pct_profit,2), 'pct_noaction':round(pct_noaction,2), 
    #     'pct_diff':round(pct_profit-pct_noaction,2)})
    # bt_result = bt_result.append(line0, ignore_index=True)

# df_temp = pd.read_csv('../data/stocklist.csv', parse_dates=True)

# bt_list.to_csv( bt_file, index=False )
# bt_list[ bt_list['st_s1']==1 ]

 .    
 1 300002.SZ [['stp', '2020-04-13']]
 .     .     .     .     .     .    
 8 300014.SZ [['stp', '2020-02-03']]
 .     .     .    
 12 300026.SZ [['stp', '2020-04-14']]

 13 300031.SZ [['stp', '2021-09-07']]
 .     .    
 16 300035.SZ [['stp', '2020-05-26'], ['stp', '2021-09-13']]
 .     .    
 19 300045.SZ [['stp', '2020-02-11'], ['stp', '2020-02-14']]

 20 300053.SZ [['stp', '2020-02-11']]

 21 300054.SZ [['stp', '2020-03-09'], ['stp', '2020-03-12']]
 .    
 23 300058.SZ [['stp', '2021-12-14']]
 .     .     .     .     .     .     .     .     .    
 33 300083.SZ [['stp', '2020-07-17']]

 34 300085.SZ [['stp', '2020-07-09']]

 35 300087.SZ [['stp', '2020-02-14'], ['stp', '2020-08-12']]
 .     .     .    
 39 300113.SZ [['stp', '2020-06-11']]
 .     .    
 42 300118.SZ [['stp', '2020-07-17']]
 .    
 44 300122.SZ [['stp', '2020-07-20']]

 45 300123.SZ [['stp', '2020-02-06'], ['stp', '2020-02-11'], ['stp', '2020-07-10'], ['stp', '2020-07-17']]
 .     .     .     .    
 50 300133.S

In [118]:
# type( bt_result.loc[0] )
# print('操作数量：', df_viewlist.shape[0])
# buy_day
# df_viewlist
bt_list


,Unnamed: 0,ts_code,symbol,name,area,industry,market,exchange,list_status,list_date,is_hs,st_s1,bt_s1
0,0,300001.SZ,300001,特锐德,山东,电气设备,创业板,SZSE,L,20091030,S,0,NaN
1,1,300002.SZ,300002,神州泰岳,北京,软件服务,创业板,SZSE,L,20091030,S,0,1.0
2,2,300003.SZ,300003,乐普医疗,北京,医疗保健,创业板,SZSE,L,20091030,S,0,NaN
3,3,300006.SZ,300006,莱美药业,重庆,化学制药,创业板,SZSE,L,20091030,S,0,NaN
4,4,300007.SZ,300007,汉威科技,河南,电器仪表,创业板,SZSE,L,20091030,S,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,286,300973.SZ,300973,立高食品,广东,食品,创业板,SZSE,L,20210415,S,0,NaN
287,287,300979.SZ,300979,华利集团,广东,服饰,创业板,SZSE,L,20210426,S,0,NaN
288,288,300981.SZ,300981,中红医疗,河北,医疗保健,创业板,SZSE,L,20210427,S,0,NaN
289,289,300999.SZ,300999,金龙鱼,上海,食品,创业板,SZSE,L,20201015,S,0,NaN


In [119]:
# 利润绘图

# x = bt_result.index
# profit = bt_result['pct_profit'].to_list()
# noaction = bt_result['pct_noaction'].to_list()
# diff = bt_result['pct_diff'].to_list()
# width = 0.2

# # tick_label = ['A', 'B', 'C']
# # tick_label = bt_result['ts_code'].to_list()
# tick_label = bt_result['ts_code'].to_list()

# plt.bar(x,         profit,   width, color="g", label='w/ QF', alpha=0.5)
# plt.bar(x+width,   noaction, width, color="b", label='no action',alpha=0.5)
# plt.bar(x+2*width, diff,     width, color="r", label='Improvement',alpha=0.5)
# # plt.bar(x, sh, width, bottom=bj)

# plt.xlabel("stock_code")
# plt.ylabel("qf performance")

# plt.xticks(x+width/2, tick_label)
# plt.legend()

# plt.show()

